# Process MPC and AstDyS-2 orbit database files

The current Minor Planet Center Orbit (MPCORB) database files are ~250Mb in size as ASCII text.  To make
them more readily accessible for plotting for the *Celestial and Stellar Mechanics* book, we process an MPCORB
snapshot into FITS BinTables with a subset of data of interest.

This takes a long time to run, but once we have the subset files, notebooks for plotting those subsets run
much more quickly.

We read the ASCII versions of the MPC database instead of using `astroquery` because the module's `MPC` class
limits us to queries of no more than 16k objects (2$^{14}$-1 to be precise), and we want a few more data
points in general. 

In [2]:
import os
import math
import numpy as np
import pandas as pd
import time

from astropy.io import fits
from astropy.time import Time

# Newton-Raphson root solver for solving Kepler's equation

from scipy.optimize import newton

# Throttle nuisance warnings

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)
warnings.filterwarnings('ignore',category=RuntimeWarning, append=True)

# useful constants

JD2000 = 2451545.0   # JD of 2000.0
daysPerYear = 365.25 # SI days per year


### Unpack MPCORB dates

This is an old-school way of coding dates as a "packed" 5-character string of numbers and letters.  See 
https://www.minorplanetcenter.net/iau/info/PackedDates.html for how dates are encoded.

This function unpacks a packed epoch string and returns the epoch as an ISO-compliant string and as
the decimal year. The MPCORB database gives epoch in the terrestrial time (TT) time system, and while use explicitly use TT scale here, practically it makes little difference for most applications related to book figures.

We use the `astropy.time` module to handle time conversions correctly.

In [3]:
def unpackMPCDate(epochStr):
    
    # century is char 1: I, J, K = 1800, 1900, 2000
    
    cc = 1800 + 100*(ord(epochStr[:1])-ord("I"))
    
    # year is chars 2 and 3
    
    yy = int(epochStr[1:3])
    ccyy = int(cc+yy)
    
    # month is char 4, 1-9,A-C = 1..12
    
    mm = ord(epochStr[3:4]) - ord("0")
    if mm > 9:
        mm = ord(epochStr[3:4]) - ord("7")
        
    # day is char 5, 1-9,A-V = 1..31
    
    dd = ord(epochStr[4:]) - ord("0")
    if dd > 9:
        dd = ord(epochStr[4:]) - ord("7")

    # convert to decimal year format
    
    isoEpoch = f'{ccyy:4d}-{mm:02d}-{dd:02d}'
    t = Time(isoEpoch,format='isot',scale='tt')
    
    return isoEpoch, t.decimalyear

## MPCORB database

The MPCORB data base (https://www.minorplanetcenter.net/iau/MPCORB/MPCORB.DAT.gz) is a fixed-width ASCII format
file originally designed to be written/read by Fortran. The file is about 75Mb compressed, 250Mb uncompressed.

We read this file using the `pandas.read_fwf()` method with an explicit `colspecs` specification to extract
the bits we care about.  The full format is described at https://www.minorplanetcenter.net/iau/info/MPOrbitFormat.html

```
mpcCols=[(0,7),(8,13),(14,19),(20,25),(26,35),(37,46),(48,57),(59,68),(70,79),(80,91),(92,103),
         (105,106),(107,116),(117,122),(123,126),(127,136),(137,142),(161,165),(166,194),(196,-1)]
```

Mapping into python index for items:
<pre>
  0 - designation
  1 - H - absolute magnitude
  2 - G - slope parameter
  3 - Epoch in packed form (see code below)
  4 - M - mean anomaly at epoch [degrees]
  5 - w - argument of perihelion J2000 [degrees]
  6 - Omega - longitude of the ascending node J2000 [degrees]
  7 - i - inclination to the ecliptic J2000 [degrees]
  8 - e - eccentricity 
  9 - n - mean daily motion [degrees/day]
 10 - semimajor axis [au]
 11 - uncertainty parameter
 12 - referencey
 13 - NObs - number of observations
 14 - NOpp - number of oppositions observed
 15 - first/last year of observation, or arc if single-opposition elements
 16 - rms (arcsec)
 17 - 4-digit hex flags
 18 - human-readable designation
 19 - date of last observation in the orbit solution
</pre>
The subset of hex codes of interest to us here (bits 0-5) are:
<pre>
   1  Atira
   2  Aten
   3  Apollo
   4  Amor
   5  Object with q < 1.665 AU
   6  Hungaria
   7  Unused or internal MPC use only
   8  Hilda
   9  Jupiter Trojan
  10  Distant object
</pre>
Richer data are encoded in the hex codes, see the format webpage for a full breakdown.

We use the compressed `MPCORB.DAT.gz` file, skipping the first 44 lines which is an explanatory header. Reading compressed vs. uncompressed with `pandas.read_fwf()` takes less than 10% longer for on-the-fly compression so we
work with the gzip-compressed `MPCORB.DAT.gz` file downloaded directly from the MPC website without modification.

### Performance

We measure the time to read the entire compressed `MPCORB.DAT.gz` file in the cell below.

For reference, on a 16Gb RAM MacMini M2 Pro running macOS 13.4.1 (Ventura) it takes a little under 17 seconds to read the entire `MPCORB` database of about 1.3 million lines of data. 

In [4]:
mpcFile = 'MPC/MPCORB.DAT.gz'

mpcCols=[(0,7),(8,13),(14,19),(20,25),(26,35),(37,46),(48,57),(59,68),(70,79),(80,91),(92,103),
         (105,106),(107,116),(117,122),(123,126),(127,136),(137,142),(161,165),(166,194),(196,-1)]

t0 = time.time()

mpcData = pd.read_fwf(mpcFile,colspecs=mpcCols,header=None,comment='#',skiprows=43,compression='gzip')

tread = time.time() - t0
print(f'{mpcFile} read in {tread:.3f} seconds')

MPC/MPCORB.DAT.gz read in 23.520 seconds


### File read checks

Some quick checks that we can read the file OK.  As of 2023 July, the `MPCORB` database contains about 1.3 million
lines of data.  This checks that we skipped the right number of rows to jump over the descriptive header at the
top of the file (currently `skiprows=43`). First line of data should be for `00001 = (1) Ceres`.  Change the `skiprows` argument in `pd.read_fwf()` above if this is not the case. We know of no robust a priori way to know in advance how may rows of header to skip.

Print a summary of the number of objects in the file and a table of the first line of data in the file.

Finally, print a table of the cumulative number of asteroids brighter than a given H absolute magnitude.

In [5]:
# Check that we counted number of rows to skip correctly.  The first entry should be Ceres.

print(f'There are {len(mpcData[0])} objects in {mpcFile}')

isoEp, decEp = unpackMPCDate(mpcData[3][0])
print(f'First entry: {mpcData[18][0]}, Epoch {isoEp} ({decEp:.6f})')

# quick check

print('\nindex     datum')
print('----------------------')
print(mpcData.loc[0,:])
print('----------------------')

# count number of asteroids brighter than a given H absolute magnitude

H = np.array(mpcData[1])

print('\nObjects brighter than H mag:')
print('-----------------')
for minH in range(5,22):
    iBright = np.where(H <= minH)[0]
    print(f'H<{minH:2d} mag: {len(iBright):7d}')  
print('-----------------')


There are 1298901 objects in MPC/MPCORB.DAT.gz
First entry: (1) Ceres, Epoch 2023-02-25 (2023.150685)

index     datum
----------------------
0         00001
1          3.33
2          0.15
3         K232P
4      17.21569
5      73.47045
6      80.26013
7      10.58635
8      0.078817
9      0.214115
10     2.767182
11            0
12    E2023-F87
13       7283.0
14          123
15    1801-2023
16         0.65
17         0000
18    (1) Ceres
19       230321
Name: 0, dtype: object
----------------------

Objects brighter than H mag:
-----------------
H< 5 mag:     117
H< 6 mag:     378
H< 7 mag:    1388
H< 8 mag:    3096
H< 9 mag:    4317
H<10 mag:    5089
H<11 mag:    5878
H<12 mag:    7440
H<13 mag:   12252
H<14 mag:   28890
H<15 mag:   79320
H<16 mag:  201807
H<17 mag:  486978
H<18 mag:  896492
H<19 mag: 1167444
H<20 mag: 1257746
H<21 mag: 1271752
-----------------


## Make subset data files

These are selected subsets of the MPCORB database to help speed-up making plots of solar system bodies that
appear in Chapters 1-x.

### Jupiter Trojan Asteroids

Make a CSV file containing the osculating orbital elements for the Jupiter Trojan asteroids in the MPCORB
database. Jupiter Trojans are designated by hex code `0009`.

In [18]:
csvFile = 'MPC_JupiterTrojans.csv'

type = np.array(mpcData[17])
iTrojans = np.where(type=='0009')[0]

name_jt = np.array(mpcData[18][iTrojans])
a_jt = np.array(mpcData[10][iTrojans])
n_jt = np.array(mpcData[9][iTrojans])
e_jt = np.array(mpcData[8][iTrojans])
i_jt = np.array(mpcData[7][iTrojans])
Omega_jt = np.array(mpcData[6][iTrojans])
peri_jt = np.array(mpcData[5][iTrojans])
M_jt = np.array(mpcData[4][iTrojans])
epStr = np.array(mpcData[3][iTrojans])
H_jt = np.array(mpcData[1][iTrojans])

# Use the epoch of the first Trojan to compute the (likely) reference epoch for all.  We handle exceptions next

isoEp,epoch = unpackMPCDate(epStr[0])
epoch_jt = epoch*np.ones(len(iTrojans))

# handle objects with a different epoch

iDiff = np.where(epStr != epStr[0])[0]
if len(iDiff) > 0:
    for i in iDiff:
        isoEp,epoch_jt[i] = unpackMPCDate(epStr[i])
        
# build the pandas data frame

trojans = {
    'Name':name_jt,
    'H':H_jt,
    'a':a_jt,
    'e':e_jt,
    'i':i_jt,
    'Omega':Omega_jt,
    'peri':peri_jt,
    'M':M_jt,
    'n':n_jt,
    'Epoch':epoch_jt
}
df = pd.DataFrame(trojans)

# Write the data frame as a CSV file

df.to_csv(csvFile,index=False)

print(f'Wrote {len(iTrojans)} orbit elements to {csvFile}')

Wrote 11873 orbit elements to MPC_JupiterTrojans.csv


### Hilda Asteroids

Make a CSV file containing the osculating orbital elements for the Hilda 3:2 resonant family asteroids in
the MPCORB database. Hildas are designated by hex code `0008`.

In [19]:
csvFile = 'MPC_Hildas.csv'

type = np.array(mpcData[17])
iHildas = np.where(type=='0008')[0]

name_h = np.array(mpcData[18][iHildas])
a_h = np.array(mpcData[10][iHildas])
n_h = np.array(mpcData[9][iHildas])
e_h = np.array(mpcData[8][iHildas])
i_h = np.array(mpcData[7][iHildas])
Omega_h = np.array(mpcData[6][iHildas])
peri_h = np.array(mpcData[5][iHildas])
M_h = np.array(mpcData[4][iHildas])
epStr = np.array(mpcData[3][iHildas])
H_h = np.array(mpcData[1][iHildas])

# Use the epoch of the first object to compute the (likely) reference epoch for all.  We handle exceptions next

isoEp,epoch = unpackMPCDate(epStr[0])
epoch_h = epoch*np.ones(len(iHildas))

# handle objects with a different epoch

iDiff = np.where(epStr != epStr[0])[0]
if len(iDiff) > 0:
    for i in iDiff:
        isoEp,epoch_h[i] = unpackMPCDate(epStr[i])

# build the pandas data frame
Hildas = {
    'Name':name_h,
    'H':H_h,
    'a':a_h,
    'e':e_h,
    'i':i_h,
    'Omega':Omega_h,
    'peri':peri_h,
    'M':M_h,
    'n':n_h,
    'Epoch':epoch_h
}
df = pd.DataFrame(Hildas)

# Write the data frame as a CSV file

df.to_csv(csvFile,index=False)

print(f'Wrote {len(iHildas)} orbit elements to {csvFile}')

Wrote 5035 orbit elements to MPC_Hildas.csv


### Main Belt asteroids and friends

Make a CSV file containing the osculating orbital elements for Main Belt asteroids and Hungarias that have
absolute magnitudes brighter than H=14.0 mag.

This is a generous sampling of the Main Belt plus the Hungarias, Hildas, and Jupiter trojan resonant groups
that are brighter than H=16 magnitudes

In [20]:
csvFile = 'MPC_MainBeltPlus.csv'

aMin = 1.5 # au
aMax = 7.0 # au
minH = 16 # magnitudes

iAster = np.where((mpcData[10] >= aMin) & (mpcData[10] <= aMax) & (mpcData[1] <= minH))[0]

name_h = np.array(mpcData[18][iAster])
H_h = np.array(mpcData[1][iAster])
a_h = np.array(mpcData[10][iAster])
e_h = np.array(mpcData[8][iAster])
i_h = np.array(mpcData[7][iAster])
Omega_h = np.array(mpcData[6][iAster])
peri_h = np.array(mpcData[5][iAster])
M_h = np.array(mpcData[4][iAster])
n_h = np.array(mpcData[9][iAster])
epStr = np.array(mpcData[3][iAster])

# Use the epoch of the first object to compute the (likely) reference epoch for all.  We handle exceptions next

isoEp,epoch = unpackMPCDate(epStr[0])
epoch_h = epoch*np.ones(len(iAster))

# handle objects with a different epoch

iDiff = np.where(epStr != epStr[0])[0]
if len(iDiff) > 0:
    for i in iDiff:
        isoEp,epoch_h[i] = unpackMPCDate(epStr[i])

# build the pandas data frame
Aster = {
    'Name':name_h,
    'H':H_h,
    'a':a_h,
    'e':e_h,
    'i':i_h,
    'Omega':Omega_h,
    'peri':peri_h,
    'M':M_h,
    'n':n_h,
    'Epoch':epoch_h
}
df = pd.DataFrame(Aster)

# Write the data frame as a CSV file

df.to_csv(csvFile,index=False)

print(f'Wrote {len(iAster)} asteroids with {aMin:.1f}<a<{aMax:.1f}au and H<{minH:.1f} to {csvFile}')

Wrote 196916 asteroids with 1.5<a<7.0au and H<16.0 to MPC_MainBeltPlus.csv


## Asteroid Histogram

For illustrating the Kirkwood Gaps in the Main Belt, we want to make a histogram of all currently known
asteroids with semimajor axes between 1.7 and 4.1 au in bins of $\Delta$a=0.004 au.


In [33]:
csvFile = 'MPC_Kirkwood.csv'

aMin = 1.7 # au
aMax = 4.1 # au
da = 0.004 # au

iAster = np.where((mpcData[10] >= aMin) & (mpcData[10] <= aMax))[0]

numAster = np.histogram(mpcData[10][iAster],np.arange(aMin,aMax+da,da))

# build the pandas data frame
astBins = {
    'a':np.arange(aMin,aMax,da),
    'count':numAster[0]
}
df = pd.DataFrame(astBins)

# Write the data frame as a CSV file

df.to_csv(csvFile,index=False)

600 601


## Comets

Read in comet orbit data from `AllCometEls.txt` from the MPC (https://www.minorplanetcenter.net/iau/MPCORB).  
We read this as a fixed-width file following the format described in 
https://www.minorplanetcenter.net/iau/info/CometOrbitFormat.html

Like reading the MPCORB file, we use the `pandas.read_fwf()` method with an explict `colspec` 
specification:
```
cometCols = [(0,4), (4,5), (5,12), (14,29), (30,39), (41,49), (51,59), (61,69), (71,79), (81,89),
            (91,95), (96,100), (102,158), (159,-1)]
```

Mapping into python index for items of interest:
<pre>
  0 - period comet number
  1 - orbit type (C,P,D)
  3 - Year, month, day of perihelion passage [TT]
  4 - q - perihelion distance [au]
  5 - e - eccentricity 
  6 - w - argument of perhelion J2000 [degrees]
  7 - Omega - longitude of the ascending node J2000 [degrees]
  8 - i - inclination to the ecliptic J2000 [degrees]
  9 - year/month/day of epoch for perturbed solutions
 10 - H - absolute magnitude
 11 - G - slope parameter
 12 - human-readable designation
 13 - reference
</pre>
This file is small enough we work with it uncompressed.

In [29]:
dataFile = 'MPC/AllCometEls.txt'

cometCols = [(0,4), (4,5), (5,12), (14,29), (30,39), (41,49), (51,59), (61,69), (71,79), (81,89),
            (91,95), (96,100), (102,158), (159,-1)]

t0 = time.time()

data = pd.read_fwf(dataFile,colspecs=cometCols,header=None,comment='#')

tread = time.time() - t0
print(f'{dataFile} read in {tread:.3f} seconds')
print(f'# comets in {dataFile}: {len(data[0])}')

MPC/AllCometEls.txt read in 0.024 seconds
# comets in MPC/AllCometEls.txt: 4517


## Proper orbit elements

Read ASCII data files of proper orbit elements from the AstDyS-2 database: https://newton.spacedys.com/astdys/

Columns:
<pre>
  0 - name
  1 - H - absolute magnitude
  2 - a_p - proper semimajor axis [au]
  3 - e_p - proper eccentricity
  4 - sin(i_p) - sine of proper inclination
  5 - n - mean motion in  [degrees/year]
  6 - g - frequency of perhelion [degrees/year] - rate of perihelion precesion
  7 - s - frequency of node [degrees/year] - rate of nodal precession
  8 - lambda  - Lyapunov exponent *1e6
  9 - integration time - [Myr]
</pre>
The `all.syn` files is uncompressed and orderly enough to read directly with `pandas.read_csv()`.  Note that
the files use `%` as the comment character. We use `low_memory=False` in order to throttle warnings from
pandas because the file mixes purely numerical and character-based names in the first column.

In [30]:
dataFile = 'AstDyS-2/all.syn'

t0 = time.time()

data = pd.read_csv(dataFile,sep=r'\s+',header=None,comment='%',low_memory=False)

tread = time.time() - t0
print(f'{dataFile} read in {tread:.3f} seconds')

print(f'# entries: {len(data[0])}')

H = np.array(data[1])

for minH in range(8,21):
    iBright = np.where(H <= minH)[0]
    print(f'{minH:2d} mag: {len(iBright):7d} asteroids')

AstDyS-2/all.syn read in 0.336 seconds
# entries: 524214
 8 mag:     165 asteroids
 9 mag:     377 asteroids
10 mag:     758 asteroids
11 mag:    1298 asteroids
12 mag:    2940 asteroids
13 mag:    9210 asteroids
14 mag:   31704 asteroids
15 mag:   94412 asteroids
16 mag:  216120 asteroids
17 mag:  380408 asteroids
18 mag:  490303 asteroids
19 mag:  522389 asteroids
20 mag:  524198 asteroids


## Orbit calculation functions

### Eccentric anomaly, $E$

The mean anomaly $M$ is related to the eccentric anomaly $E$ and eccentricity $e$ by Kepler's equation:
 > $M = E - e\sin(E)$

Kepler's equation cannot be solved analytially to compute $E$ given $M$ and $e$ from the orbit elements, so
we solve it numerically using the Newton-Raphson method.  

Define a function $f(E)$
 > $f(E) = E - e\sin(E) - M$

with derivative
 > $\frac{dfE}{dE} = 1 - e\cos(E)$

And solve for $f(E)=0$ given $M$ and $e$.  We will use the `scipy.optimize.newton()` function to solve
for the roots using the Newton-Raphson method.

### True anomaly, $\nu$

Compute the true anomaly $\nu$ given the eccentric anomaly $E$ and orbit eccentricity $e$, using a form that
is numerically safe when E is near $\pm\pi$:
\begin{equation}
  \nu = E + 2\arctan\left(\frac{\beta\sin E}{1-\beta\cos E}\right)
\end{equation}
where
\begin{equation}
  \beta = \frac{e}{1+(1-e^2)^{1/2}}
\end{equation}

### orbital elements to ecliptic (x,y,z) at a given epoch

...


In [31]:
# Kepler's equation - assumes M and E in radians

def kepler(E,M,e):
    return E - e*np.sin(E) - M

# Derivative of Kepler's equation, E in radians

def kepler_deriv(E,M,e):
    return 1 - e*np.cos(E)

# Compute the eccentric anomaly - M must be in radians

def eccAnomaly(M,e):
    E = newton(kepler, M, kepler_deriv, args=(M,e))
    return E

# Compute the true anomaly - E must be in radians

def trueAnomaly(E,e):
    beta = e/(1.0+np.sqrt(1-e*e))
    top = beta*np.sin(E)
    bot = 1 - beta*np.cos(E)
    nu = E + 2.0*np.arctan2(top,bot)
    return nu

# Compute ecliptic (x,y,z) coordinates at epochXY given orbit elements (a,e,n,M) at elEpoch

def orbXYZ(a,e,n,M,i,Omega,peri,elEpoch,epochXY):
    dT = epochXY - elEpoch # years
    dM = np.radians(dT*n*365.25) # radians
    Mep = M + dM # mean anomaly at epochXY in radians
    
    # eccentric anomaly, E, at epochXY in radians
    
    Er = eccAnomaly(Mep,e)
    
    # true anomaly, nu, at epochXY in radians
    
    nu = trueAnomaly(Er,e)
    
    # heliocentric radius in au
    
    r = a*(1-e*e)/(1 + e*np.cos(nu))
    
    # orbit plane (x,y), perihelion is toward +x
    
    xorb = r*np.cos(nu)
    yorb = r*np.sin(nu)

    x,y,z = orbToEcliptic(xorb,yorb,Omega,i,peri)

    return x,y,z

# Convert orbital plane (x,y) to ecliptic plane (x,y,z)
    
def orbToEcliptic(xorb,yorb,Omega,i,peri):

    # Euler matrix - angles must be in radians
    
    m_xx = np.cos(peri)*np.cos(Omega) - np.sin(peri)*np.cos(i)*np.sin(Omega)
    m_xy = np.cos(peri)*np.sin(Omega) + np.sin(peri)*np.cos(i)*np.cos(Omega)
    m_xz = np.sin(peri)*np.sin(i)
    
    m_yx = -np.sin(peri)*np.cos(Omega) - np.cos(peri)*np.cos(i)*np.sin(Omega)
    m_yy = -np.sin(peri)*np.sin(Omega) + np.cos(peri)*np.cos(i)*np.cos(Omega)
    m_yz =  np.cos(peri)*np.sin(i)
    
    # because zorb=0 by definition, we don't need these
    #m_zx =  np.sin(i)*np.sin(Omega)
    #m_zy = -np.sin(i)*np.cos(Omega)
    #m_zz =  np.cos(i)
    
    # compute ecliptic XYZ
    
    xEcl = xorb*m_xx + yorb*m_yx # + zorb*m_zx
    yEcl = xorb*m_xy + yorb*m_yy # + zorb*m_zy
    zEcl = xorb*m_xz + yorb*m_yz # + zorb*m_zz
    
    return xEcl,yEcl,zEcl

# trace a full orbit in ecliptic cartesian coordinats

def traceOrb(a,e,i,Omega,peri):
    nu = np.linspace(0.0,2.*np.pi,501)
    r = a*(1-e*e)/(1+e*np.cos(nu))
    xorb = r*np.cos(nu)
    yorb = r*np.sin(nu)
    
    x,y,z = orbToEcliptic(xorb,yorb,np.radians(Omega),np.radians(i),np.radians(peri))

    return x,y,z

# rotate ecliptic (x,y) coordinates into the co-rotating heliocentric frame of planet at (xP,yP)

def corotXY(x,y,xP,yP):
    dX = x - xP
    dY = y - yP

    rP = np.sqrt(xP*xP + yP*yP) # heliocentric distance
    theta = np.arctan2(yP,xP)   # rotation angle

    xcr =  dX*np.cos(theta) + dY*np.sin(theta) + rP
    ycr = -dX*np.sin(theta) + dY*np.cos(theta)

    return xcr,ycr

## Jupiter Trojans XYZ



In [35]:
orbFile = 'JupiterTrojans.csv'

data = pd.read_csv(orbFile)

elEpoch = np.array(data['Epoch']) # epoch of the orbit elements in years

a = np.array(data['a']) # semimajor axis
e = np.array(data['e']) # eccentricity
n = np.array(data['n']) # mean motion in degrees/day

# convert angular orbit element angles to radians

i = np.radians(np.array(data['i']))  # inclination of the orbital plane
Omega = np.radians(np.array(data['Omega'])) # longitude of the ascending node
peri = np.radians(np.array(data['peri']))   # argument of periastron
M = np.radians(np.array(data['M']))         # mean anomaly at epoch

print(f'read {len(a)} object orbit elements')

# Epoch of the solar system plot to make

viewDate = '2023-05-02'
t = Time(viewDate,format='isot',scale='tt') # astropy time object
viewEpoch = t.decimalyear
print(f'Computing positions for Epoch {viewDate} ({viewEpoch:.5f})')

# compute XYZ positions

x,y,z = orbXYZ(a,e,n,M,i,Omega,peri,elEpoch,viewEpoch)

print(f'x: min={np.min(x):.3f} max={np.max(x):.3f} au')
print(f'y: min={np.min(y):.3f} max={np.max(y):.3f} au')
print(f'z: min={np.min(z):.3f} max={np.max(z):.3f} au')


read 11873 object orbit elements
Computing positions for Epoch 2023-05-02 (2023.33151)
x: min=-5.242 max=5.279 au
y: min=-7.152 max=7.224 au
z: min=-3.162 max=3.684 au
